In [1]:
import os
import json
import torch
import utils
import numpy as np
from tqdm import tqdm
from collections import Counter

In [2]:
#Load the cui data
# cuidata, _ = utils.split_cui_present_absent(True)
DATA_PATH = "P:\ORD_Singh_201911038D\Justin\Stroke_Notes_13OCT21"
with open("sids_to_present_cuis.json",'r') as infile:
    cuidata = json.load(infile)

In [3]:
#print(list(cuidata)[0]) # print the first document ID
print(cuidata[list(cuidata)[0]].keys()) # Just to show the data fields in each document
print(cuidata[list(cuidata)[0]]['cuis'][:10]) # show the first 10 cuis of this specific document

dict_keys(['cuis', 'semtyp'])
['C0008031', 'C0015031', 'C4718442', 'C2707412', 'C0398266', 'C3244243', 'C0917798', 'C0013227', 'C4715678', 'C4715678']


In [4]:
# We will get training pairs. One is the context term the next is the target term.
trainingpairs = []
vocab = dict()
frequency = Counter()
idx = 0

# This could absolutely be accomplished in an alternative / faster way but it's sufficiently fast for now
for doc in tqdm(cuidata):
    cuis = cuidata[doc]['cuis']
    cuiids = []
    for x in cuis:
        if x not in vocab.keys():
            vocab.update({x:idx})
            idx+=1
        cuiids += [vocab[x]]
    for i in range(len(cuiids)-1):
        pair = cuiids[i:i+2]
        if len(pair) < 2:
            continue
        if pair[0] != pair[1]:
            trainingpairs += [pair] #no self references
            frequency.update(pair)
print(trainingpairs[:3])
print(len(trainingpairs))
print(len(vocab))
assert np.all(np.asarray([x for x in vocab.values()]) == np.arange(len(vocab)))
# We'll use an embedding dimension of 50 to start
EMBEDDING_DIM = 100

100%|███████████████████████████████████████████████████████████████████████████| 29865/29865 [00:30<00:00, 986.56it/s]

[[0, 1], [1, 2], [2, 3]]
2879508
101991


In [5]:
frequency = np.asarray([values for key,values in frequency.items()])**0.75
frequency /= np.linalg.norm(frequency, ord=1)
frequency = torch.from_numpy(frequency)
samplingids = torch.arange(0, len(frequency))

In [6]:
class CUIEmbeddingModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CUIEmbeddingModel, self).__init__()
        self.dim = embedding_dim
        self.in_embeddings = torch.nn.Embedding(vocab_size, embedding_dim, sparse=True)
        self.out_embeddings = torch.nn.Embedding(vocab_size, embedding_dim, sparse=True)
        
    def init(self):
        self.in_embeddings.weight.data.uniform_(-0.5/self.dim, 0.5/self.dim) # scaled by dimensionality to control initial norm.
        self.out_embeddings.weight.data.uniform(-0, 0) # all 0s
        
    def forward(self, inputs, targets, negatives):
        inembed = self.in_embeddings(inputs)
        outembed = self.out_embeddings(targets)
        pos_score = torch.sum(torch.mul(inembed, outembed), dim=1)
        pos_score = torch.nn.functional.logsigmoid(pos_score)
        negembed = self.out_embeddings(negatives)
        neg_score = torch.bmm(negembed, inembed.unsqueeze(2)).squeeze()
        neg_score = torch.nn.functional.logsigmoid(-1*neg_score)
        # skip gram negative sampling
        return -1 * (torch.sum(pos_score)+torch.sum(neg_score))
    

In [7]:
losses = []
model = CUIEmbeddingModel(len(vocab), EMBEDDING_DIM)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
NEGSAMPLES = 2

In [8]:
#Test batching (also gives you an idea of theoretical max throughput in a way)
BATCH_SIZE=2000
for batchidx in tqdm(np.arange(0, len(trainingpairs), BATCH_SIZE), total = len(trainingpairs)//BATCH_SIZE):
    data = trainingpairs[batchidx:batchidx+BATCH_SIZE]
    context_idxs = torch.tensor(data)

1440it [00:01, 769.57it/s]                                                                                             


In [17]:
def nearest_embedding_search(cui):
    #C0948008 = ischemic stroke
    for i in zip(*torch.topk(torch.nn.functional.cosine_similarity(model.in_embeddings.weight.data[vocab[cui]].view(1,-1), 
                                                                   model.in_embeddings.weight.data), 10, largest=True)):
        try:
            print(f'{i[0]:0.3f}\t{cuitranslate[id2vocab[int(i[1])]]}')
        except:
            print("key error", id2vocab[int(i[1])])

In [10]:
id2vocab = {value:key for key,value in vocab.items()}
with open(os.path.join(DATA_PATH, "cuitranslate.json"),'r') as infile:
    cuitranslate = json.load(infile)

In [11]:
print(list(id2vocab.items())[0:10])
print(list(cuitranslate.items())[0:10])
print(cuitranslate["C0000005"])

[(0, 'C0008031'), (1, 'C0015031'), (2, 'C4718442'), (3, 'C2707412'), (4, 'C0398266'), (5, 'C3244243'), (6, 'C0917798'), (7, 'C0013227'), (8, 'C4715678'), (9, 'C0014806')]
[('C0000005', '(131)I-Macroaggregated Albumin'), ('C0000039', '1,2-dipalmitoylphosphatidylcholine'), ('C0000052', '1,4-alpha-Glucan Branching Enzyme'), ('C0000074', '1-Alkyl-2-Acylphosphatidates'), ('C0000084', '1-Carboxyglutamic Acid'), ('C0000096', '1-Methyl-3-isobutylxanthine'), ('C0000097', '1-Methyl-4-phenyl-1,2,3,6-tetrahydropyridine'), ('C0000098', '1-Methyl-4-phenylpyridinium'), ('C0000102', '1-Naphthylamine'), ('C0000103', '1-Naphthylisothiocyanate')]
(131)I-Macroaggregated Albumin


In [18]:
trainingpairs = torch.LongTensor(trainingpairs)
for epoch in range(50):
    total_loss = 0
    shuffler = torch.randperm(trainingpairs.shape[0])
    trainingpairs = trainingpairs[shuffler].view(trainingpairs.size())
    with tqdm(np.arange(0, len(trainingpairs), BATCH_SIZE), desc=f'Epoch {epoch+1}', total = len(trainingpairs)//BATCH_SIZE) as progress: #goes one example at a time
        for batchidx in progress:
            data = trainingpairs[batchidx:batchidx+BATCH_SIZE]
            inputs = torch.cat((data[:, 0], data[:,1])) #we'll go bidirectional; usually not done I suppose
            targets = torch.cat((data[:, 1], data[:,0]))#doubles the batch size
            negatives = samplingids[frequency.multinomial(num_samples=inputs.shape[0]*NEGSAMPLES, replacement=True)].reshape(inputs.shape[0], NEGSAMPLES)
            optimizer.zero_grad()
            loss = model.forward(inputs, targets, negatives)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            progress.set_postfix(loss=f"{loss.item():3.3f}")
        losses.append(total_loss)
        progress.set_postfix(loss=f"{total_loss:3.3f}")
    nearest_embedding_search('C0948008')
    
print(losses)


Epoch 1: 1440it [07:18,  3.28it/s, loss=16465.070]                                                                     


1.000	Ischemic stroke
0.390	CANCER WORKUP
0.389	Pancrease
0.388	Non-Palpable
key error microtuition
0.382	Fracture of bone of face
0.381	Lymphadenopathy
0.377	Intravascular ultrasound of artery
0.377	Spinal canal stenosis
0.376	PreNatal Vitamins Plus Tablet


Epoch 2: 1440it [10:13,  2.35it/s, loss=14865.372]                                                                     


1.000	Ischemic stroke
0.497	Acute Cerebrovascular Accidents
0.431	work restrictions decision pending further evaluation
0.422	Poor judgement
0.411	alteplase
0.399	histidine
0.395	Dysarthria
0.393	Plateau
0.390	PreNatal Vitamins Plus Tablet
0.382	Orifice area:Area:Pt:Aortic valve:Qn:US.continuity.VTI+Area


Epoch 3: 1440it [06:51,  3.50it/s, loss=13869.231]                                                                     


...

1.000	Ischemic stroke
0.930	Acute Cerebrovascular Accidents
0.927	Acute ischemic stroke subtype
0.905	WEAKNESS RESIDUAL
0.895	Stroke, Lacunar
0.891	Transient Ischemic Attack
0.891	CVA DISTRIBUTION MCA
0.887	Thalamic infarction
0.886	CANCER WORKUP
0.882	Intracranial Hemorrhage


Epoch 49: 1440it [07:06,  3.37it/s, loss=6090.204]                                                                     


1.000	Ischemic stroke
0.935	Acute Cerebrovascular Accidents
0.923	Acute ischemic stroke subtype
0.908	WEAKNESS RESIDUAL
0.901	CANCER WORKUP
0.897	Stroke, Lacunar
0.895	CVA DISTRIBUTION MCA
0.894	Thalamic infarction
0.889	Left hemiparesis
0.888	Right hemiparesis


Epoch 50: 1440it [07:09,  3.35it/s, loss=5829.967]                                                                     


1.000	Ischemic stroke
0.934	Acute Cerebrovascular Accidents
0.927	Acute ischemic stroke subtype
0.907	WEAKNESS RESIDUAL
0.894	CVA DISTRIBUTION MCA
0.893	Stroke, Lacunar
0.889	Thalamic infarction
0.888	Transient Ischemic Attack
0.886	CANCER WORKUP
0.881	Intracranial Hemorrhage
[48117001.0, 34838506.712890625, 30136581.215820312, 27425331.391601562, 25559103.729492188, 24140458.036132812, 23013291.181640625, 22087911.190429688, 21303551.436523438, 20624172.674804688, 20006899.734375, 19459065.2421875, 18958038.936523438, 18490251.908203125, 18061518.086914062, 17674138.061523438, 17319280.380859375, 16983358.043945312, 16664058.073242188, 16357197.025390625, 16083402.198242188, 15807204.088867188, 15563606.02734375, 15319388.051757812, 15087667.051757812, 14865657.7578125, 14693355.289550781, 14465770.283691406, 14276719.106445312, 14097417.866210938, 13922525.47265625, 13762162.331054688, 13594302.790039062, 13442185.420898438, 13293640.6796875, 13153114.493164062, 13020411.931640625, 1

In [19]:
torch.save(model.state_dict(), 'cui_model_6_MAR_2022.pt')

In [20]:
for _, i in enumerate(losses):
    if losses[_-1] < i: #if loss is always monitonically decreasing, should print 0 only
        print(_)

0


In [21]:
with open("cui_vectors_6_MAR_2022.json",'w') as outfile:
    json.dump({key:model.in_embeddings.weight.data[value].tolist() for key,value in vocab.items()}, outfile)

In [15]:
# ToDo: Build document vectors from CUI vector components
docvectors = np.zeros((len(cuidata), EMBEDDING_DIM))
for i,doc in tqdm(enumerate(cuidata), total=len(cuidata)):
    for cui in cuidata[doc]['cuis']:
        docvectors[i] += model.in_embeddings.weight.data[vocab[cui]].numpy()


100%|███████████████████████████████████| 29865/29865 [01:16<00:00, 391.28it/s]


In [21]:
# ToDo: Build document vectors from CUI vector components
docvectors = np.zeros((len(cuidata), EMBEDDING_DIM))
for i,doc in tqdm(enumerate(cuidata), total=len(cuidata)):
    tmpvecs = np.zeros(EMBEDDING_DIM)
    for cui in cuidata[doc]['cuis']:
        tmpvecs += model.in_embeddings.weight.data[vocab[cui]].numpy()
    docvectors[i] = tmpvecs / len(cuidata[doc]['cuis'])

100%|███████████████████████████████████| 29865/29865 [01:07<00:00, 439.69it/s]


In [22]:
with open('average_document_vectors_28_FEB_2022.json','w') as outfile:
    json.dump({x:list(y) for x,y in zip(cuidata.keys(), docvectors)}, outfile)

In [27]:
'''
To load the saved pytorch model:

model = CUIEmbeddingModel(len(vocab), EMBEDDING_DIM) #vocab and embedding dim must match what the model was trained on
model.load_state_dict(torch.load(cui_model_15_NOV_2021.pt))
model.eval() #set to evaluation mode
''';

In [32]:
nearest_embedding_search('C0742946')

1.000	CVA ETIOLOGY HEMORRHAGIC ISCHEMIC
0.847	Acute Cerebrovascular Accidents
0.836	Ischemic stroke
0.829	Left hemiparesis
0.829	Structure of middle cerebral artery
0.822	Thalamic infarction
0.821	Transient Ischemic Attack
0.820	Evaluation
0.815	Acute ischemic stroke subtype
0.815	transient ischemic attack without residual deficits


In [33]:
import xgboost